<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Visualizaci%C3%B3n_Integrada_Instrumentos_Version_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MONITOREO INSTRUMENTACIÓN PRESAS

#**Carga Masiva de datos Crudos**

In [2]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={'AS175_20250608.csv': {'metadata': {'name': 'AS175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: PUNTOS_FIJOS_MD (CSV)


,FECHA,MARGEN,INSTRUMENTO,DELTA_NORTE_[M],DELTA_ESTE_[M],DELTA_COTA_[M],DISTANCIA_[M],DISTANCIA_(MM),AZIMUT_REF._AL_NORTE,TASA_NORTE_(MM/DIA),TASA_ESTE_(MM/DIA),TASA_COTA_(MM/DIA),TASA_DISTANCIA_(MM/DIA)
0,04/10/2021,Derecha,PF01 PRIMA,0.000333,-0.000667,0.003333,0.000745,0.745356,296.565051,0.041667,-0.083333,0.416667,0.093169
1,10/10/2021,Derecha,PF01 PRIMA,-0.000667,-0.000667,0.004333,0.000943,0.942809,225.000000,-0.047619,-0.047619,0.309524,0.067344
2,14/10/2021,Derecha,PF01 PRIMA,0.003333,0.003333,0.003333,0.004714,4.714045,45.000000,0.185185,0.185185,0.185185,0.261891
3,18/10/2021,Derecha,PF01 PRIMA,0.002333,0.002333,0.000333,0.003300,3.299832,45.000000,0.106061,0.106061,0.015152,0.149992
4,27/10/2021,Derecha,PF01 PRIMA,-0.000667,-0.000667,0.005333,0.000943,0.942809,225.000000,-0.021505,-0.021505,0.172043,0.030413


#Visualización Integrada de Multiples Instrumentos

In [5]:
import pandas as pd

try:
    import plotly.graph_objects as go
    import plotly.colors as pc
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import importlib.util
    import os
except ImportError:
    raise ImportError("Ejecuta: pip install plotly ipywidgets")

# === WIDGETS COMUNES ===
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos"],
    value="Puntos Fijos", description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
output = widgets.Output()
output_guardar = widgets.Output()

# === WIDGETS PUNTOS FIJOS ===
margen_dropdown = widgets.Dropdown(description="Margen:")
punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
# === WIDGETS PIEZÓMETROS ELÉCTRICOS ===
progresiva_dropdown = widgets.Dropdown(description="Progresiva:")
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:")

# === WIDGETS COMUNES DE GRÁFICA ===
variable_dropdown = widgets.Dropdown(description="Variable:")
anio_dropdown = widgets.Dropdown(description="Año:")
estilo_dropdown = widgets.Dropdown(
    options=["Curvas suaves (spline)", "Líneas rectas", "Puntos", "Líneas + Puntos",
             "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"],
    value="Curvas suaves (spline)", description="Estilo:"
)
tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500), description="Tamaño:"
)
grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor:"
)
paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly, description="Paleta:"
)
ruta_text = widgets.Text(value="grafica", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:"
)
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")

# === FUNCIONES AUXILIARES ===

def obtener_df(instr):
    origen = origen_dropdown.value
    if instr == "Puntos Fijos":
        return {
            "Margen Izquierda (MI)": datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"],
            "Margen Derecha (MD)": datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
        }
    elif instr == "Piezómetros Eléctricos":
        df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
        return df.copy()
    return None

def actualizar_controles(*args):
    clear_output(wait=True)
    display(HTML("<h3 style='color:#1866a3'>Visualización Interactiva</h3>"))
    display(widgets.HBox([instrumento_dropdown, origen_dropdown]))

    if instrumento_dropdown.value == "Puntos Fijos":
        datasets = obtener_df("Puntos Fijos")
        if not datasets:
            print("⚠️ No hay datos cargados para Puntos Fijos.")
            return
        margen_dropdown.options = list(datasets.keys())
        df = datasets[margen_dropdown.value].copy()
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        variables = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
        variable_dropdown.options = variables
        variable_dropdown.value = variables[0] if variables else None
        punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique())
        anios = sorted(df['FECHA'].dt.year.dropna().unique())
        anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
        anio_dropdown.value = "Todos"

        display(widgets.HBox([margen_dropdown, punto_dropdown, variable_dropdown, anio_dropdown]))
    else:
        df = obtener_df("Piezómetros Eléctricos")
        if df.empty:
            print("⚠️ No hay datos para Piezómetros Eléctricos.")
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique())
        progresiva_dropdown.value = progresiva_dropdown.options[0]
        actualizar_piezometros()
        variables = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']]
        variable_dropdown.options = variables
        variable_dropdown.value = variables[0] if variables else None
        anios = sorted(df['FECHA'].dt.year.dropna().unique())
        anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
        anio_dropdown.value = "Todos"

        display(widgets.HBox([progresiva_dropdown, piezometro_dropdown, variable_dropdown, anio_dropdown]))

    display(widgets.HBox([estilo_dropdown, tamanio_dropdown]))
    display(widgets.HBox([grosor_dropdown, paleta_dropdown]))
    display(boton)
    display(output)
    display(widgets.HBox([formato_dropdown, ruta_text, boton_guardar]))
    display(output_guardar)

def actualizar_piezometros(*args):
    df = obtener_df("Piezómetros Eléctricos")
    if df.empty:
        piezometro_dropdown.options = []
        return
    prog = progresiva_dropdown.value
    piezos = sorted(df[df['PROGRESIVA'] == prog]['PIEZOMETRO'].dropna().unique())
    piezometro_dropdown.options = ["Todos"] + piezos
    piezometro_dropdown.value = "Todos"

def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
        if instrumento_dropdown.value == "Puntos Fijos":
            df = obtener_df("Puntos Fijos")[margen_dropdown.value].copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            var = variable_dropdown.value
            df = df.dropna(subset=['FECHA', 'INSTRUMENTO', var])
            if anio_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_dropdown.value)]
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value]
            grupo = df['INSTRUMENTO']
        else:
            df = obtener_df("Piezómetros Eléctricos").copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
            var = variable_dropdown.value
            df = df.dropna(subset=['FECHA', 'PIEZOMETRO', var])
            if anio_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_dropdown.value)]
            grupo = df['PIEZOMETRO']

        if df.empty:
            print("⚠️ No hay datos para graficar.")
            return

        fig = go.Figure()
        color_map = {g: paleta_dropdown.value[i % len(paleta_dropdown.value)] for i, g in enumerate(grupo.unique())}
        modo = estilo_dropdown.value
        for g in grupo.unique():
            datos = df[grupo == g]
            mode_map = {
                "Curvas suaves (spline)": ("lines", "spline"),
                "Líneas rectas": ("lines", "linear"),
                "Puntos": ("markers", None),
                "Líneas + Puntos": ("lines+markers", "linear"),
                "Área apilada": ("lines", "linear"),
                "Área + Líneas": ("lines", "linear"),
                "Área + Líneas + Puntos": ("lines+markers", "linear")
            }
            m, shape = mode_map[modo]
            fill = "tozeroy" if "Área" in modo else None
            stack = "one" if modo == "Área apilada" else None
            fig.add_trace(go.Scatter(
                x=datos['FECHA'], y=datos[var], mode=m, name=g,
                line_shape=shape, fill=fill, stackgroup=stack,
                line=dict(width=grosor_dropdown.value, color=color_map[g]),
                marker=dict(color=color_map[g])
            ))
        ancho, alto = tamanio_dropdown.value
        fig.update_layout(
            width=ancho, height=alto,
            title=f"{instrumento_dropdown.value}: {var} vs Fecha",
            xaxis_title="Fecha", yaxis_title=var, hovermode="x unified"
        )
        fig.show()

def guardar_grafica(b=None):
    with output_guardar:
        clear_output(wait=True)
        ext = formato_dropdown.value
        ruta = ruta_text.value
        if not ruta.endswith(ext):
            ruta += ext
        if 'fig' not in globals():
            print("⚠️ Primero genera una gráfica.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("⚠️ Instala kaleido: %pip install -U kaleido")
                    return
                fig.write_image(ruta)
            elif ext == ".html":
                fig.write_html(ruta)
            print(f"✅ Gráfica guardada en: {os.path.abspath(ruta)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# === EVENTOS ===
instrumento_dropdown.observe(actualizar_controles, names='value')
origen_dropdown.observe(actualizar_controles, names='value')
margen_dropdown.observe(actualizar_controles, names='value')
progresiva_dropdown.observe(actualizar_piezometros, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# === MOSTRAR UI INICIAL ===
actualizar_controles()


Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Output()

In [6]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# === Selectores Comunes ===
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos"],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:"
)

# === Selectores Comunes de Gráfica ===
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)", description="Estilo:"
)
grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor:"
)
tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500), description="Tamaño:"
)
paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly, description="Paleta:"
)

# === Selectores por tipo de instrumento ===
# Puntos Fijos
margen_dropdown = widgets.Dropdown(description="Margen:")
punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
variable_pf_dropdown = widgets.Dropdown(description="Variable:")
anio_pf_dropdown = widgets.Dropdown(description="Año:")

# Piezómetros Eléctricos
progresiva_dropdown = widgets.Dropdown(description="Progresiva:")
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_pe_dropdown = widgets.Dropdown(description="Variable:")
anio_pe_dropdown = widgets.Dropdown(description="Año:")

# === Botones y salida ===
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:"
)
output = widgets.Output()
output_guardar = widgets.Output()

# === Mostrar/Ocultar Selectores según Instrumento ===
def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value
    for widget in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
                   progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown]:
        widget.layout.display = 'none'

    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    else:
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()

# === Datos cargados externos (asegúrate de tener estos dicts ya cargados) ===
# datos_csv = {"puntos_fijos_mi": ..., "puntos_fijos_md": ..., "piezometros_electricos": ...}
# datos_xlsx = {igual}

# === Actualizar selectores PF ===
def actualizar_opciones_pf():
    origen = origen_dropdown.value
    df_mi = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
    df_md = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
    datasets = {
        "Margen Izquierda (MI)": df_mi.copy(),
        "Margen Derecha (MD)": df_md.copy()
    }
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets:
        return

    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')

    columnas_excluir = ['FECHA', 'INSTRUMENTO', 'MARGEN']
    variables = [col for col in df.select_dtypes(include='number').columns if col not in columnas_excluir]
    variable_pf_dropdown.options = variables
    if variables:
        variable_pf_dropdown.value = variables[0]

    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique())
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_pf_dropdown.options = ["Todos"] + [str(a) for a in anios]

# === Actualizar selectores PE ===
def actualizar_opciones_pe():
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty:
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique())
    progresiva_dropdown.value = progresiva_dropdown.options[0]

    actualizar_piezometros_pe()
    columnas_excluir = ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']
    variables = [c for c in df.select_dtypes(include='number').columns if c not in columnas_excluir]
    variable_pe_dropdown.options = variables
    if variables:
        variable_pe_dropdown.value = variables[0]
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_pe_dropdown.options = ["Todos"] + [str(a) for a in anios]

def actualizar_piezometros_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    prog = progresiva_dropdown.value
    piezos = sorted(df[df['PROGRESIVA'] == prog]['PIEZOMETRO'].dropna().unique())
    piezometro_dropdown.options = ["Todos"] + piezos

# === Gráfica ===
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)

    if instrumento_dropdown.value == "Puntos Fijos":
        origen = origen_dropdown.value
        df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
        if margen_dropdown.value == "Margen Derecha (MD)":
            df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
        df = df.copy()
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        df = df.dropna(subset=['FECHA', 'INSTRUMENTO', variable_pf_dropdown.value])

        if anio_pf_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
        if punto_dropdown.value != "Todos":
            df = df[df['INSTRUMENTO'] == punto_dropdown.value]
        titulo = f"{margen_dropdown.value}: {variable_pf_dropdown.value}"

        instrumento_col = 'INSTRUMENTO'
        variable = variable_pf_dropdown.value

    else:
        df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
        df = df.copy()
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
        if piezometro_dropdown.value != "Todos":
            df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
        if anio_pe_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
        df = df.dropna(subset=['FECHA', variable_pe_dropdown.value])
        titulo = f"{progresiva_dropdown.value} – {variable_pe_dropdown.value}"

        instrumento_col = 'PIEZOMETRO'
        variable = variable_pe_dropdown.value

    if df.empty:
        print("⚠️ No hay datos para graficar.")
        return

    fig = go.Figure()
    elementos = sorted(df[instrumento_col].unique())
    colores = {pf: paleta_dropdown.value[i % len(paleta_dropdown.value)] for i, pf in enumerate(elementos)}
    modo = {
        "Curvas suaves (spline)": ("lines", "spline"),
        "Líneas rectas": ("lines", "linear"),
        "Puntos": ("markers", None),
        "Líneas + Puntos": ("lines+markers", "linear"),
        "Área apilada": ("lines", "linear"),
        "Área + Líneas": ("lines", "linear"),
        "Área + Líneas + Puntos": ("lines+markers", "linear")
    }
    modo_graf, line_shape = modo[estilo_dropdown.value]
    fill = "tozeroy" if "Área" in estilo_dropdown.value else None
    stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

    for elem in elementos:
        d = df[df[instrumento_col] == elem]
        fig.add_trace(go.Scatter(
            x=d['FECHA'], y=d[variable],
            mode=modo_graf,
            name=elem,
            line=dict(width=grosor_dropdown.value, color=colores[elem]),
            marker=dict(color=colores[elem]),
            fill=fill, stackgroup=stackgroup,
            line_shape=line_shape
        ))

    ancho, alto = tamanio_dropdown.value
    fig.update_layout(
        width=ancho, height=alto,
        title=titulo,
        xaxis_title="Fecha",
        yaxis_title=variable,
        hovermode="x unified",
        legend_title=instrumento_col
    )
    fig.show()

# === Guardar gráfica ===
def guardar_grafica(b=None):
    with output_guardar:
        clear_output()
        ext = formato_dropdown.value
        ruta = ruta_text.value
        if not ruta.endswith(ext):
            ruta += ext
        if 'fig' not in globals():
            print("⚠️ Primero debes generar la gráfica.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instala 'kaleido':\n%pip install -U kaleido")
                    return
                fig.write_image(ruta)
            elif ext == ".html":
                fig.write_html(ruta)
            print(f"✅ Guardado: {os.path.abspath(ruta)}")
        except Exception as e:
            print("❌ Error:", e)

# === Observadores ===
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# === Mostrar interfaz ===
display(HTML("<h3 style='color:#1866a3'>Visualización Interactiva</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))
display(widgets.HBox([margen_dropdown, punto_dropdown, progresiva_dropdown, piezometro_dropdown]))
display(widgets.HBox([variable_pf_dropdown, variable_pe_dropdown, anio_pf_dropdown, anio_pe_dropdown]))
display(widgets.HBox([estilo_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown]))
display(boton)
display(output)
display(widgets.HBox([formato_dropdown, ruta_text, boton_guardar]))
display(output_guardar)

# Inicializar visibilidad
actualizar_controles_visibles()


Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Output()